# Итоговый проект (часть 3: бонус) #
### Елизавета Клыкова, БКЛ181 ###
**Описание:** программа собирает тексты по фандому "Мистер Робот", опубликованные на сайте https://ficbook.net и лемматизирует их. Затем на основании этих текстов и русских текстов, полученных с сайта https://archiveofourown.org, строится векторная модель, после чего модели сравниваются.

Сначала включим проверку на PEP-8 и импортируем все необходимые для работы модули.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import collections
import gensim
import matplotlib.pyplot as plt
import nltk
import os
import re
import requests
import string
import sqlite3
import unicodedata
import warnings
from bs4 import BeautifulSoup
from gensim.models import KeyedVectors
from html import unescape
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from pymorphy2 import MorphAnalyzer
from string import punctuation
from tqdm.auto import tqdm
from wordcloud import WordCloud
%matplotlib inline
morph = MorphAnalyzer()
p = punctuation + '«»…–'
session = requests.session()
warnings.filterwarnings("ignore")

### Пункт 1. Получение данных ###
Нам понадобятся три функции: для обхода страницы поиска; для получения данных о каждой работе на странице поиска; для получения полного текста работы со страницы отдельной работы.

В качестве уникального fanfic_id будем использовать число из ссылки на полный текст работы.

In [3]:
def parse_first_level_info(f):
    ff = {}
    # эти элементы не могут отсутствовать, поэтому проверка не нужна
    ff['full_link'] = 'https://ficbook.net' + f.find(
        'a', {'class': 'visit-link'}).attrs['href']
    ff['fanfic_id'] = (re.search('([0-9])+', ff['full_link'])).group()
    ff['title'] = f.find('a', {'class': 'visit-link'}).text
    ff['author'] = f.find('span', {'class': 'author'}).text.strip()
    ff['theme'] = f.find(
        'span', {'class': 'direction'}).attrs['title'].split('—')[0].strip()
    ff['description'] = f.find(
        'div', {'class': 'fanfic-description-text'}).text.strip()

    full_info = f.find('dl', {'class': 'info'}).text
    info = re.sub(r'\n(Фэндом:|Пэйринг |Рейтинг:|Размер:|Статус:|Метки:)',
                  r'*\1', full_info).split('*')

    fandoms = pairings = rating = size_ = status_ = ''
    for i in info:
        if i.startswith('Фэндом:'):
            fandoms = [re.sub(r'Фэндом:|\(кроссовер\)',
                              '', fm).strip() for fm in i.split(',')]
        if i.startswith('Пэйринг '):
            pairings = [pr.replace('Пэйринг и персонажи:',
                                   '').strip() for pr in i.split(',')]
        if i.startswith('Рейтинг:'):
            rating = i.replace('Рейтинг:', '').strip()
        if i.startswith('Размер:'):
            size_ = ', '.join([' '.join(
                s.split())for s in i.replace('Размер:', '').split(',')])
        if i.startswith('Статус:'):
            status_ = i.replace('Статус:', '').strip()

    if fandoms:
        ff['fandoms'] = ', '.join(fandoms)
    else:
        ff['fandoms'] = fandoms

    if pairings:
        ff['pairings'] = ', '.join(pairings)
    else:
        ff['pairings'] = pairings

    ff['rating'], ff['size_'], ff['status_'] = rating, size_, status_

    # меток может не быть, поэтому проверяем; если их нет, задаем ('None')
    try:
        tags = (re.sub('Показать|спойлеры', '', f.find('dd', {
            'class': 'tags'}).text.replace('Метки:', ''))).strip().split('\n')
    except AttributeError:
        tags = ['None']
    ff['tags'] = tags

    # иногда авторы отключают возможность оценивания работ, поэтому проверка:
    try:
        ff['grade'] = f.find('span', {'class': 'value'}).text
    except AttributeError:
        ff['grade'] = 'Not Stated'

    return ff

Теперь функция, получающая информацию со страниц отдельных работ. Отсюда нужно получить полный текст работы. Проблема заключается в том, что если в работе больше одной части, нужно получать ссылки на все части и проходить уже по ним. 

Проверяя, сколько частей в работе, будем получать полный текст, если часть одна, и список ссылок на все части, если больше одной.

In [4]:
def parse_second_level_info(ff):
    url_one = ff['full_link']
    page = session.get(url_one).text
    soup = BeautifulSoup(page, 'html.parser')

    # из поля "Размер" получим информацию о количестве частей
    n_parts = ff['size_'].split(',')[-1].strip()
    if n_parts == '1 часть':  # если одна часть, просто вытащим текст
        ff['full_text'] = soup.find('div', {'id': 'content'}).text
    else:
        links_list = soup.find_all('ul', {'class': 'table-of-contents'})
        links = []
        for a_link in links_list:
            link = 'https://ficbook.net' + a_link.find(
                'a', {'class': 'visit-link'}).attrs['href']
            links.append(link)
        try:
            ff['full_text'] = parse_third_level_info(links)
        except Exception as e:
            print(e)

    return ff

Наконец, функция, которая получает части текста со страниц глав и возвращает полный текст работы:

In [5]:
def parse_third_level_info(links):
    full_text = []
    for link in links:
        page = session.get(link).text
        soup = BeautifulSoup(page, 'html.parser')
        txt = soup.find('div', {'id': 'content'}).text
        full_text.append(txt)
    return '\n'.join(full_text)

Теперь объявим функцию, получающую информацию обо всех работах на странице.

In [6]:
def get_nth_page(page_number):
    url = f'https://ficbook.net/fanfiction/movies_and_tv_series/mister_robot?p={page_number}.html'
    page = session.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    fanfics = soup.find_all('article', {'class': 'block'})
    blocks = []
    for f in fanfics:
        try:
            blocks.append(parse_first_level_info(f))
        except Exception as e:
            print(e)
    result = []
    for b in blocks:
        try:
            res = parse_second_level_info(b)
            result.append(res)
        except Exception as e:
            print(e)
    return result

2:80: E501 line too long (98 > 79 characters)


### Пункт 2. Создание базы данных ###
Добавим в уже существующую базу данных "eaklykova_final.db" три таблицы:

* таблицу fb_fanfics с общей информацией о работе (колонки: id, id работы, название, автор, список фандомов, список пэйрингов, рейтинг, размер, статус, направленность, количество оценок, краткое описание, ссылка на страницу работы, полный текст работы)
* таблицу fb_tags с метками (колонки: id метки, название метки)
* таблицу fb_fanfic_to_tag, связывающую таблицы 1 и 2 связью один-ко-многим (колонки: id, id работы, id метки).

In [7]:
con = sqlite3.connect('eaklykova_final.db')
cur = con.cursor()

In [8]:
def create_database():
    cur.execute("""
    CREATE TABLE IF NOT EXISTS fb_fanfics
    (id INTEGER PRIMARY KEY, fanfic_id text, title text, author text,
    fandoms text, pairings text, rating text, size_ text, status_ text,
    theme text, grade text, description text, full_link text,
    full_text text)
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS fb_tags
    (id INTEGER PRIMARY KEY, tag text)""")

    cur.execute("""
    CREATE TABLE IF NOT EXISTS fb_fanfic_to_tag
    (id INTEGER PRIMARY KEY AUTOINCREMENT, id_fanfic int, id_tag int)
    """)

    con.commit()
    con.close()

In [9]:
create_database()

### Пункт 3. Запись в базу ###
Для каждой работы проверим, что ее уникального fanfic_id еще нет в базе. Если условие выполнено, запишем теги этой работы по одному в словарь db_tags, в котором содержатся все уникальные теги. Каждый тег сопоставляется с индексом, равным длине списка db_tags + 1. Теги с индексами записываются в таблицу tags.

В свою очередь, каждой работе приписывается индекс, равный длине списка seen_fanfics + 1. Этот индекс записывается в таблицу fanfics наряду с остальной информацией (fanfic_id, название, автор и т.д.).

Наконец, в таблицу связей записываются id работы и id тегов, которые ей соответствуют.

В список errors записываются ошибки, возникающие при записи работ в базу. При каждом перезапуске программы туда кладется информация о работах, которые не были записаны в базу, потому что уже в ней есть.

In [10]:
def write_to_db(blocks):
    errors = []
    for block in blocks:
        if block['fanfic_id'] not in seen_fanfics:
            seen_fanfics.add(block['fanfic_id'])
            tags = []
            for tag in block['tags']:
                if tag in db_tags:
                    tags.append(db_tags[tag])
                else:
                    db_tags[tag] = len(db_tags) + 1
                    tags.append(db_tags[tag])
                    cur.execute('INSERT INTO fb_tags VALUES (?, ?)', (
                        len(db_tags), tag))
                    con.commit()

            f_id = len(seen_fanfics)
            cur.execute('''
            INSERT INTO fb_fanfics VALUES (?, ?, ?, ?, ?, ?, ?, ?,
            ?, ?, ?, ?, ?, ?)''',
                        (f_id, block['fanfic_id'], block['title'],
                         block['author'], block['fandoms'], block['pairings'],
                         block['rating'], block['size_'], block['status_'],
                         block['theme'], block['grade'], block['description'],
                         block['full_link'], block['full_text'])
                        )

            tags = [(f_id, t) for t in tags]
            cur.executemany(
                '''INSERT INTO fb_fanfic_to_tag (id_fanfic, id_tag)
                VALUES (?, ?)''',
                tags)
            con.commit()

        else:
            errors.append('Работа с id ' + block['fanfic_id']
                          + ' уже есть в базе')
    return errors

Подключаемся к базе, получаем список тегов, которые уже встретились раньше, из таблицы tags. Список уже просмотренных работ получаем из таблицы fanfics и кладем в переменную seen_fanfics.

In [11]:
con = sqlite3.connect('eaklykova_final.db')
cur = con.cursor()
cur.execute('SELECT tag, id FROM fb_tags')

db_tags = {}
for name, idx in cur.fetchall():
    db_tags[name] = idx

cur.execute('SELECT fanfic_id FROM fb_fanfics')
seen_fanfics = set(i[0] for i in cur.fetchall())

Собираем в общую функцию:

In [12]:
def run_all(n_pages):
    errors = []
    for i in tqdm(range(n_pages)):
        try:
            err = write_to_db(get_nth_page(i+1))
            errors.extend(err)
        except Exception as e:
            print(e)
    return errors

Запустим программу на всех страницах (их 17).

In [13]:
errors = run_all(17)

In [14]:
con.close()

### Пункт 4. Создание модели ###
#### Лемматизация, подготовка файлов ####

In [15]:
db = sqlite3.connect('eaklykova_final.db')
cur = db.cursor()
cur.execute(
    '''ALTER TABLE fb_fanfics ADD COLUMN lem_texts text''')
db.commit()

In [16]:
cur.execute('SELECT fanfic_id, full_text FROM fb_fanfics')
fb_texts = cur.fetchall()
fb_text_list = [list(text) for text in fb_texts]

Токенизируем русские тексты с помощью NLTK и лемматизируем их с помощью pymorphy, разделив предожения переносами строки.

In [30]:
punct = string.punctuation  # стандартная пунктуация
other_punct = ['``', "\'\'", '...', '--', 'https', '–',
               '—', '«', '»', '“', '”', '’', '***', '…', '']
for text in tqdm(fb_text_list):
    sents = sent_tokenize(text[1].lower())
    lemm_sents = []
    for sent in sents:
        tokens = word_tokenize(sent)
        lemmas = [morph.parse(t.strip('…'))[0].normal_form
                  for t in tokens
                  if t not in punct and t not in other_punct]
        if lemmas:
            lemm_sents.append(' '.join(lemmas))
    lem_text = '\n'.join(lemm_sents)
    text.append(lem_text)

Соединим тексты, полученные с сайта https://archiveofourown.org и записанные в файл 'rus_model.txt', с текстами с https://ficbook.net. Запишем все тексты в файл 'fb_model.txt'.

In [31]:
def make_file_for_model(file1, file2, texts):
    with open(file1, encoding='utf-8') as f1:
        rus_texts = f1.read()
    with open(file2, 'w', encoding='utf-8') as f2:
        all_texts = []
        for text in texts:
            all_texts.append(text[-1])
        big_fb_text = ''.join(all_texts)
        full_text = rus_texts + '\n' + big_fb_text
        f2.write(full_text)

In [32]:
make_file_for_model('rus_model.txt',
                    'fb_model.txt', fb_text_list)

Запишем лемматизированные тексты в базу данных.

In [33]:
def lemm_texts_to_db(text_list):
    for text in text_list:
        cur.execute('''UPDATE fb_fanfics SET lem_texts = ?
        WHERE fanfic_id = ?''', (text[-1], text[0],))
    db.commit()

In [34]:
lemm_texts_to_db(fb_text_list)

In [35]:
db.close()

#### Создание модели ####

In [36]:
def create_model(filename, model_path):
    f = filename
    data = gensim.models.word2vec.LineSentence(f)
    fm = gensim.models.Word2Vec(
        data, size=300, window=5, min_count=5, iter=50)
    fm.init_sims(replace=True)
    fm.wv.save_word2vec_format(model_path, binary=True)
    return len(fm.wv.vocab)

In [37]:
fb_len = create_model('fb_model.txt', 'fb_model.bin')
print('Кол-во слов в полной русской модели:', fb_len)

Кол-во слов в полной русской модели: 6627


Загрузим все три получившиеся модели и сравним их.

In [39]:
eng_m = KeyedVectors.load_word2vec_format('eng_model.bin', binary=True)
rus_m = KeyedVectors.load_word2vec_format('rus_model.bin', binary=True)
fb_m = KeyedVectors.load_word2vec_format('fb_model.bin', binary=True)

In [40]:
def most_sim_w(model, word):
    if word in model.wv:
        title = 'Соседи слова "' + word + '":\n'
        similar_w = model.wv.most_similar(positive=[word], topn=5)
        sim_ws = [w[0] + ' (' + str(w[1]) + ')'
                  for w in similar_w]
        result = title + '\n'.join(sim_ws)
    else:
        result = 'К сожалению, выбранного слова нет в модели.'
    return result

In [41]:
print(most_sim_w(eng_m, 'elliot') + '\n')
print(most_sim_w(rus_m, 'эллиот') + '\n')
print('Полная модель.', most_sim_w(fb_m, 'эллиот'))

Соседи слова "elliot":
tyrell (0.7937356233596802)
he (0.7520555257797241)
leon (0.6383670568466187)
robot (0.5559099912643433)
him (0.5541512966156006)

Соседи слова "эллиот":
он (0.7281099557876587)
тайрелла (0.7055686116218567)
шейла (0.6241317391395569)
мужчина (0.5810321569442749)
боль (0.5765253305435181)

Полная модель. Соседи слова "эллиот":
он (0.6010484099388123)
тайрелла (0.47550326585769653)
я (0.4288792908191681)
ирвинга (0.3620915412902832)
хакер (0.3534810543060303)


In [42]:
print(most_sim_w(eng_m, 'robot') + '\n')
print(most_sim_w(rus_m, 'робот') + '\n')
print('Полная модель.', most_sim_w(fb_m, 'робот'))

Соседи слова "robot":
elliot (0.5559099912643433)
tyrell (0.5529675483703613)
perry (0.5231907963752747)
leon (0.5110501050949097)
jovonovich (0.5056238174438477)

Соседи слова "робот":
норма (0.6695342659950256)
глупый (0.6450705528259277)
i (0.6145631670951843)
приходиться (0.6097466349601746)
дрожать (0.5666431188583374)

Полная модель. Соседи слова "робот":
грэм (0.46817851066589355)
уэлик (0.458244264125824)
велик (0.4218754172325134)
малёк (0.4015697240829468)
-да (0.39449357986450195)


Тайрелл и разные вариации фамилии "Уэллик" - это один из центральных персонажей, который некоторое время работает вместе с Эллиотом и Мистером Роботом, поэтому неудивительно, что его имя появляется так часто.

In [43]:
print(most_sim_w(eng_m, 'corporation') + '\n')
print(most_sim_w(rus_m, 'корпорация') + '\n')
print('Полная модель.', most_sim_w(fb_m, 'корпорация'))

Соседи слова "corporation":
conglomerate (0.6791448593139648)
government (0.5076230764389038)
corporate (0.47649556398391724)
capitalist (0.4698414206504822)
academic (0.4658206105232239)

Соседи слова "корпорация":
абсолютно (0.8531997203826904)
компьютерный (0.8173185586929321)
нечего (0.8075254559516907)
крутой (0.7634763717651367)
неважно (0.7530659437179565)

Полная модель. Соседи слова "корпорация":
мировой (0.5352795124053955)
технология (0.5063686370849609)
технический (0.49849390983581543)
зло (0.4869730770587921)
правительство (0.46851664781570435)


Слова, выданные первой и третьей моделями, согласуются с темой сериала - это сериал-антиутопия о борьбе группы хакеров с крупнейшей американской корпорацией E-Corp.

In [44]:
def most_sim_ws(model, words):
    word1 = words[0]
    word2 = words[1]
    if word1 in model.wv and word2 in model.wv:
        title = 'Соседи слов "' + word1 + '" и "' + word2 + '":\n'
        similar_w = model.wv.most_similar(positive=words, topn=5)
        sim_ws = [w[0] + ' (' + str(w[1]) + ')' for w in similar_w]
        result = title + '\n'.join(sim_ws)
    else:
        result = 'Одного или обоих слов нет в модели.'
    return result

In [45]:
print(most_sim_ws(eng_m, ['start', 'end']) + '\n')
print(most_sim_ws(rus_m, ['начало', 'конец']) + '\n')
print('Полная модель.', most_sim_ws(fb_m, ['начало', 'конец']))

Соседи слов "start" и "end":
begin (0.47154802083969116)
speed (0.39863741397857666)
continue (0.32734882831573486)
wake (0.29056844115257263)
resume (0.2869288921356201)

Соседи слов "начало" и "конец":
когда-то (0.6717181205749512)
традиция (0.6271486282348633)
дело (0.6247795820236206)
рассвет (0.6193835735321045)
сей (0.6053721904754639)

Полная модель. Соседи слов "начало" и "конец":
русло (0.3601152300834656)
начальник (0.33164262771606445)
доходить (0.33126792311668396)
папа (0.326962411403656)
e-corp. (0.3231198489665985)


In [46]:
print(most_sim_ws(eng_m, ['good', 'bad']) + '\n')
print(most_sim_ws(rus_m, ['хороший', 'плохой']) + '\n')
print('Полная модель.', most_sim_ws(fb_m, ['хороший', 'плохой']))

Соседи слов "good" и "bad":
great (0.5352451801300049)
terrible (0.5308564901351929)
amazing (0.4878973960876465)
real (0.44867995381355286)
stupid (0.44414809346199036)

Соседи слов "хороший" и "плохой":
факт (0.7684264183044434)
худой (0.7443269491195679)
чего-то (0.7266480922698975)
объяснять (0.7126061916351318)
попробовать (0.7089781761169434)

Полная модель. Соседи слов "хороший" и "плохой":
интересный (0.4282445013523102)
простой (0.36124318838119507)
когда-либо (0.3521459400653839)
согласиться (0.34911420941352844)
нормальный (0.3458293676376343)


Здесь хорошо видно, насколько точна английская модель в противовес обеим русским. Однако более полная русская модель все же справилась немного лучше.

In [47]:
print(most_sim_ws(eng_m, ['memory', 'thought']) + '\n')
print(most_sim_ws(rus_m, ['память', 'мысль']) + '\n')
print('Полная модель.', most_sim_ws(fb_m, ['память', 'мысль']))

Соседи слов "memory" и "thought":
nightmare (0.43525034189224243)
feeling (0.42609623074531555)
emotion (0.42093342542648315)
consciousness (0.40808865427970886)
guilt (0.4007054567337036)

Соседи слов "память" и "мысль":
разум (0.6978389024734497)
попкорн (0.6957900524139404)
душа (0.6858916878700256)
душить (0.6797796487808228)
успокаиваться (0.675955057144165)

Полная модель. Соседи слов "память" и "мысль":
воспоминание (0.4764951765537262)
голов (0.43954193592071533)
прошлое (0.4044570326805115)
пространство (0.3916492760181427)
разум (0.3888927400112152)


Здесь превосходство более полной русской модели над неполной еще более заметно: результаты начинают напоминать слова, выданные английской моделью.

In [48]:
def semantic_proportion(model, word1, word2, word3):
    try:
        comment = word1 + ' + ' + word2 + ' - ' + word3 + ' = '
        res = model.wv.most_similar(
            positive=[word1, word2], negative=[word3], topn=1)[0][0]
        result = comment + res
    except KeyError:
        result = 'Одного или нескольких слов нет в модели.'
    return result

In [49]:
print(semantic_proportion(eng_m, 'man', 'son', 'woman') + '\n')
print(semantic_proportion(rus_m, 'мужчина', 'сын', 'женщина') + '\n')
print('Полная модель:', semantic_proportion(
    fb_m, 'мужчина', 'сын', 'женщина'))

man + son - woman = father

мужчина + сын - женщина = работа

Полная модель: мужчина + сын - женщина = привязанность


In [50]:
print(semantic_proportion(eng_m, 'year', 'time', 'long') + '\n')
print(semantic_proportion(rus_m, 'год', 'время', 'долгий') + '\n')
print('Полная модель:', semantic_proportion(
    fb_m, 'год', 'время', 'долгий'))

year + time - long = day

год + время - долгий = день

Полная модель: год + время - долгий = день


Пример выше - чистая случайность, но очень приятная :)

**Вывод:** английская модель гораздо точнее, что объясняется бОльшим объемом текстов, использованных для ее обучения. Более полная русская модель лишь незначительно точнее, чем исходная.